## <i> 0/1 KNAPSACK USING BRANCH AND BOUND </i>

In [10]:
def knapsack_branch_and_bound(weights, values, capacity):
    n = len(weights)
    max_value = [0]
    possible_knapsack = []

    def calculate_upper_bound(curr_weight, curr_value, index):
        # Calculate the upper bound for the remaining items
        remaining_value = curr_value
        remaining_weight = curr_weight

        for i in range(index, n):
            remaining_value += values[i]
            remaining_weight += weights[i]

        return remaining_value if remaining_weight <= capacity else 0

    def knapsack_util(curr_weight, curr_value, index, knapsack):
        nonlocal max_value

        if index == n:
            if curr_value > max_value[0]:
                max_value[0] = curr_value
                possible_knapsack.clear()
                possible_knapsack.append((curr_weight, curr_value, knapsack.copy()))
            elif curr_value == max_value[0]:
                possible_knapsack.append((curr_weight, curr_value, knapsack.copy()))
            return

        # Calculate upper bound to determine if further exploration is needed
        upper_bound = calculate_upper_bound(curr_weight, curr_value, index)

        if upper_bound > max_value[0]:
            # Include the current item and explore
            knapsack.append(index)
            knapsack_util(curr_weight + weights[index], curr_value + values[index], index + 1, knapsack)
            knapsack.pop()

        # Exclude the current item and explore
        knapsack_util(curr_weight, curr_value, index + 1, knapsack)

    knapsack_util(0, 0, 0, [])
    return max_value[0], possible_knapsack

weights = [2, 3, 6, 5]
values = [2, 3, 6, 5]
capacity = 8

max_value, possible_knapsacks = knapsack_branch_and_bound(weights, values, capacity)
print(f"The maximum value for the 0/1 Knapsack problem (with Branch and Bound) is: {max_value}")
print("Possible knapsacks with maximum value:")
for knapsack in possible_knapsacks:
    print(f"Weights: {knapsack[0]}, Values: {knapsack[1]}, Items: {knapsack[2]}")

The maximum value for the 0/1 Knapsack problem (with Branch and Bound) is: 5
Possible knapsacks with maximum value:
Weights: 5, Values: 5, Items: [3]


## @@ TRIAL

In [12]:
class Item:
    def __init__(self, weight, value, index):
        self.weight = weight
        self.value = value
        self.index = index
        self.ratio = value / weight

def knapsack_branch_and_bound_all(capacity, weights, values):
    n = len(weights)
    items = [Item(weights[i], values[i], i) for i in range(n)]
    items.sort(key=lambda x: x.ratio, reverse=True)

    def bound(i, current_weight, current_value):
        if current_weight >= capacity:
            return 0
        total_value = current_value
        total_weight = current_weight

        while i < n and total_weight + items[i].weight <= capacity:
            total_weight += items[i].weight
            total_value += items[i].value
            i += 1

        if i < n:
            total_value += (capacity - total_weight) * items[i].ratio

        return total_value

    def branch_and_bound_recursive(i, current_weight, current_value, selected_items):
        nonlocal max_value, results

        if i == n:
            if current_value >= max_value:
                if current_value > max_value:
                    max_value = current_value
                    results = []

                results.append((current_weight, current_value, selected_items.copy()))
            return

        if current_weight + items[i].weight <= capacity:
            bound_value = bound(i + 1, current_weight + items[i].weight, current_value + items[i].value)
            if bound_value >= max_value:
                branch_and_bound_recursive(i + 1, current_weight + items[i].weight, current_value + items[i].value, selected_items + [items[i].index])

        bound_value = bound(i + 1, current_weight, current_value)
        if bound_value >= max_value:
            branch_and_bound_recursive(i + 1, current_weight, current_value, selected_items)

    max_value = 0
    results = []
    branch_and_bound_recursive(0, 0, 0, [])

    # Print all possible answers
    for result in results:
        print("Selected items:", [items[i].index for i in result[2]])
        print("Total weight:", result[0])
        print("Total value:", result[1])
        print()

# Example usage:
capacity = 8
weights = [2, 3, 6, 5]
values = [2, 3, 6, 5]

knapsack_branch_and_bound_all(capacity, weights, values)


Selected items: [0, 3]
Total weight: 7
Total value: 7



In [1]:
from queue import PriorityQueue

class Job:
    def __init__(self, job_id, deadline, penalty):
        self.job_id = job_id
        self.deadline = deadline
        self.penalty = penalty

    def __lt__(self, other):
        return self.penalty > other.penalty 

def job_sequencing_with_deadlines(jobs, max_deadline):
    job_queue = PriorityQueue()
    for job in jobs:
        job_queue.put(job)

    job_sequence = [-1] * max_deadline
    
    while not job_queue.empty():
        job = job_queue.get()
        for d in range(job.deadline - 1, -1, -1):
            if job_sequence[d] == -1:
                job_sequence[d] = job.job_id
                break

    return job_sequence

jobs = [Job("a", 2, 100), Job("b", 1, 19), Job("c", 2, 27), Job("d", 1, 25), Job("e", 3, 15)]
max_deadline = max(job.deadline for job in jobs)
scheduled_jobs = job_sequencing_with_deadlines(jobs, max_deadline)
print("Job sequence:", scheduled_jobs)


Job sequence: ['c', 'a', 'e']


In [2]:
from queue import PriorityQueue

class Job:
    def __init__(self, job_id , penalty, deadline,  processing_time):
        self.job_id = job_id
        self.deadline = deadline
        self.penalty = penalty
        self.processing_time = processing_time
        self.completion_time = -1  # Initialize completion time to -1

    def __lt__(self, other):
        return self.penalty > other.penalty 

def job_sequencing_with_deadlines(jobs, max_deadline):
    job_queue = PriorityQueue()
    for job in jobs:
        job_queue.put(job)

    job_sequence = [-1] * max_deadline
    total_penalty = 0
    current_time = 0
    
    while not job_queue.empty():
        job = job_queue.get()
        for d in range(job.deadline - 1, -1, -1):
            if job_sequence[d] == -1:
                job_sequence[d] = job.job_id
                job.completion_time = current_time + job.processing_time
                total_penalty += job.penalty
                current_time = job.completion_time
                break

    return job_sequence, total_penalty

# Example usage:
jobs = [
    Job("a", 5,1,1),
    Job("b", 10,3,2),
    Job("c", 6,2,1),
    Job("d", 3,1,1),

]

max_deadline = max(job.deadline for job in jobs)
scheduled_jobs, total_penalty = job_sequencing_with_deadlines(jobs, max_deadline)
print("Job sequence:", scheduled_jobs)
print("Total Penalty:", total_penalty)


Job sequence: ['a', 'c', 'b']
Total Penalty: 21
